# NLP - 2018
## Assignment-2
### Atishay Jain 16110024

## Tasks 1 and 2

In [4]:
from nltk.tokenize import sent_tokenize
import random

f = file("AliceInWonderland",'r')
s = f.read()
l = ["‘",'“','”','—',';',',',':','[',']','(',')','\t', '*', '\n', '-', '   ','  ']
for i in l:
    s = s.replace(i,' ')
    
s = s.replace("’",'')
s = s.lower()
sent = sent_tokenize(s)
for i in xrange(len(sent)):
    sent[i] = "<s> " + sent[i].rstrip('.!?') + " </s>"

random.shuffle(sent)
split = int(len(sent)*0.8)
train = sent[:split]
test = sent[split:]




## Task 3

In [19]:
def model_create(sent):
    u_MLE = {}
    b_MLE = {}
    t_MLE = {}
    q_MLE = {}
    u_c = {}
    b_c = {}
    t_c = {}
    q_c = {}
    tot_u = 0
    tot_b = 0
    tot_t = 0
    tot_q = 0

    for i in sent:
        l = i.split()
        for j in range(len(l)):
            tot_u += 1
            if l[j] in u_c:
                u_c[l[j]] += 1
            else:
                u_c[l[j]] = 1
            if j<(len(l)-1):
                tot_b += 1
                if l[j] + ' ' + l[j+1] in b_c:
                    b_c[l[j] + ' ' + l[j+1]] += 1
                else:
                    b_c[l[j] + ' ' + l[j+1]] = 1
            if j<(len(l)-2):
                tot_t += 1
                if l[j] + ' ' + l[j+1] + ' ' + l[j+2] in t_c:
                    t_c[l[j] + ' ' + l[j+1] + ' ' + l[j+2]] += 1
                else:
                    t_c[l[j] + ' ' + l[j+1] + ' ' + l[j+2]] = 1
            if j<(len(l)-3):
                tot_q += 1
                if l[j] + ' ' + l[j+1] + ' ' + l[j+2] + ' ' + l[j+3] in q_c:
                    q_c[l[j] + ' ' + l[j+1] + ' ' + l[j+2] + ' ' + l[j+3]] += 1
                else:
                    q_c[l[j] + ' ' + l[j+1] + ' ' + l[j+2] + ' ' + l[j+3]] = 1


    for i in q_c:
        q_MLE[i] = q_c[i]/float(t_c[i.split()[0] + ' ' + i.split()[1] + ' ' + i.split()[2]])

    for i in t_c:
        t_MLE[i] = t_c[i]/float(b_c[i.split()[0] + ' ' + i.split()[1]])

    for i in b_c:
        b_MLE[i] = b_c[i]/float(u_c[i.split()[0]])

    for i in u_c:
        u_MLE[i] = u_c[i]/float(tot_u)    
    
    return u_MLE, b_MLE, t_MLE, q_MLE, u_c, b_c, t_c, q_c, tot_u, tot_b, tot_t, tot_q


u_MLE, b_MLE, t_MLE, q_MLE, u_c, b_c, t_c, q_c, tot_u, tot_b, tot_t, tot_q = model_create(sent)

print ("For the entire corpus")
print ("N-gram \t\t Existing n-grams \t\t Total Possible n-grams" )
print ("Unigrams\t "+str(len(u_MLE))+" \t\t\t\t "+ str(len(u_MLE)))
print ("Bigrams\t\t "+str(len(b_MLE))+" \t\t\t\t "+ str(len(u_MLE)**2))
print ("Trigrams\t "+str(len(t_MLE))+" \t\t\t\t "+ str(len(u_MLE)**3))
print ("Quadgrams\t "+str(len(q_MLE))+" \t\t\t\t "+ str(len(u_MLE)**4))


For the entire corpus
N-gram 		 Existing n-grams 		 Total Possible n-grams
Unigrams	 2622 				 2622
Bigrams		 14459 				 6874884
Trigrams	 22228 				 18025945848
Quadgrams	 23731 				 47264030013456


## Task 4

In [20]:
#4
import numpy as np
import collections
import math


def Generator(mn):
    s = '<s>'
    u_mle = collections.OrderedDict(u_MLE)
    if mn == 1:
        
        s = ''
        while True:
            
            w = np.random.multinomial(1,u_mle.values())
            for i in range(len(w)):
                if w[i] == 1:
                    w = i
                    break
            w = u_mle.keys()[w]
            if w == '<s>':
                continue
            
            if w == '</s>':
                break
            s += ' ' + w
        s = s.lstrip().capitalize()
        
    elif mn == 2:

        w1 = '<s>'
        while True:
            l = {}
            for i in b_MLE:
                if i.split()[0] == w1:
                    l[i] = b_MLE[i]
                    
            l = collections.OrderedDict(l)
            w = np.random.multinomial(1,l.values())
            for i in range(len(w)):
                if w[i] == 1:
                    w = i
                    break
                    
            
            w = l.keys()[w].split()[1]
            if w == '<s>':
                continue
            
            if w == '</s>':
                break
            s += ' ' + w
            w1 = w
        s = s[4:].capitalize()
            
    elif mn == 3:
        w1 = '<s>'
        w2 = '<s>'
        while (w2 == '<s>' or w2== '</s>'):
            l = {}
            for i in b_MLE:
                if i.split()[0] == w1:
                    l[i] = b_MLE[i]
                    
            l = collections.OrderedDict(l)
            w = np.random.multinomial(1,l.values())
            for i in range(len(w)):
                if w[i] == 1:
                    w = i
                    break
            w2 = l.keys()[w].split()[1]

        s += ' ' + w2

        while True:
            l = {}
            for i in t_MLE:
                if i.split()[0] == w1 and i.split()[1] == w2:
                    l[i] = t_MLE[i]

            l = collections.OrderedDict(l)
            w = np.random.multinomial(1,l.values())
            for i in range(len(w)):
                if w[i] == 1:
                    w = i
                    break
                    
            
            w = l.keys()[w].split()[2]
            
            if w == '<s>':
                continue
            
            if w == '</s>':
                break
            s += ' ' + w
            w1 = w2
            w2 = w
        s = s[4:].capitalize()
        
    elif mn == 4:
        
        w1 = '<s>'
        w2 = '<s>'
        w3 = '<s>'
        
        while (w2 == '<s>' or w2== '</s>'):
            l = {}
            for i in b_MLE:
                if i.split()[0] == w1:
                    l[i] = b_MLE[i]
                    
            l = collections.OrderedDict(l)
            w = np.random.multinomial(1,l.values())
            for i in range(len(w)):
                if w[i] == 1:
                    w = i
                    break
            w2 = l.keys()[w].split()[1]

            
        
        while (w3 == '<s>' or w3== '</s>'):
            l = {}
            for i in t_MLE:
                if i.split()[0] == w1 and i.split()[1] == w2:
                    l[i] = t_MLE[i]
                    
            l = collections.OrderedDict(l)
            w = np.random.multinomial(1,l.values())
            for i in range(len(w)):
                if w[i] == 1:
                    w = i
                    break
            w3 = l.keys()[w].split()[2]

        s += ' ' + w2 + ' ' + w3        
        
        while True:
            l = {}
            for i in q_MLE:
                if i.split()[0] == w1 and i.split()[1] == w2 and i.split()[2] == w3:
                    l[i] = q_MLE[i]

            l = collections.OrderedDict(l)
            w = np.random.multinomial(1,l.values())
            for i in range(len(w)):
                if w[i] == 1:
                    w = i
                    break
                    
            
            w = l.keys()[w].split()[3]
            
            if w == '<s>':
                continue
            
            if w == '</s>':
                break
            s += ' ' + w
            w1 = w2
            w2 = w3
            w3 = w
            
        s = s[4:].capitalize()
        
        
    return s



def Probability(sent,mn):
    p = 0
    if mn == 1:
        l = sent.split()
        if len(l) == 0:
            p = '-inf'
        for i in l:
            if i not in u_MLE:
                p = "-inf"
                break
            p+= math.log(u_MLE[i])
            
    elif mn == 2:
        l = ['<s>'] + sent.split() + ['</s>']
            
        for i in range(len(l[:-1])):
            if l[i] + ' ' + l[i+1]  not in b_MLE:
                p = "-inf"
                break
            p+= math.log(b_c[l[i] + ' ' + l[i+1]]/float(tot_b))
        if len(l) == 2:
            p = '-inf'
            
    elif mn == 3:
        l = ['<s>'] + sent.split() + ['</s>']
        if len(l)<3:
            p = "-inf"
        for i in range(len(l[:-2])):
            if l[i] + ' ' + l[i+1] + ' ' + l[i+2] not in t_MLE:
                p = "-inf"
                break
            p+= math.log(t_c[l[i] + ' ' + l[i+1] + ' ' + l[i+2]]/float(tot_t))
        
    elif mn == 4:
        l = ['<s>'] + sent.split() + ['</s>']
        if len(l)<4:
            p = "-inf"
        for i in range(len(l[:-3])):
            if l[i] + ' ' + l[i+1] + ' ' + l[i+2] + ' ' + l[i+3] not in q_MLE:
                p = "-inf"
                break
            p+= math.log(q_c[l[i] + ' ' + l[i+1] + ' ' + l[i+2] + ' ' + l[i+3]]/float(tot_q))
        
        
    return p

s = Generator(4)
p = Probability(s.lower(),4)
#print s
#print p

## For tasks 5,6 and 7 I will use the training dataset as well

In [52]:
tr_u_MLE = {}
tr_b_MLE = {}
tr_u_c = {}
tr_b_c = {}
tr_tot_u = 0
tr_tot_b = 0
for i in train:
        l = i.split()
        for j in range(len(l)):
            tr_tot_u += 1
            if l[j] in tr_u_c:
                tr_u_c[l[j]] += 1
            else:
                tr_u_c[l[j]] = 1
            if j<(len(l)-1):
                tr_tot_b += 1
                if l[j] + ' ' + l[j+1] in tr_b_c:
                    tr_b_c[l[j] + ' ' + l[j+1]] += 1
                else:
                    tr_b_c[l[j] + ' ' + l[j+1]] = 1

        for i in tr_b_c:
            tr_b_MLE[i] = tr_b_c[i]/float(tr_u_c[i.split()[0]])

        for i in tr_u_c:
            tr_u_MLE[i] = tr_u_c[i]/float(tr_tot_u)    
    

## Task 5

In [53]:
#5 Part 1 (Calculating the counts after one add smoothing)

add_one_count = {}
for i in u_c:
    for j in u_c:
        if i == '</s>':  #because a bigram cannot start with </s> (end of sentence token)
            break
        if i + ' ' + j not in tr_b_c:
            if i not in tr_u_c: #to take care of unigrams which are there in the corpus but not in the training set
                add_one_count[i + ' ' + j] = (1.0/(len(u_MLE)))
            else:
                add_one_count[i + ' ' + j] = (1.0/(len(u_MLE) + tr_u_c[i]))*tr_u_c[i]
        else:
            add_one_count[i + ' ' + j] = ((1.0 + tr_b_c[i + ' ' + j])/(len(u_MLE) + tr_u_c[i]))*tr_u_c[i]


In [36]:
#5 Part 2 (Giving exmaples for drastic changes)

print ("Three examples of bigrams whose change in count were drastic")
print ("1. <s>,i")
print ("Original count = " + str(b_c["<s> i"]))
print ("Count after smoothing = " + str(add_one_count["<s> i"]))
print
print ("2. said the")
print ("Original count = " + str(b_c["said the"]))
print ("Count after smoothing = " +  str(add_one_count["said the"]))
print
print ("3. of the")
print ("Original count = " + str(b_c["of the"]))
print ("Count after smoothing = " + str(add_one_count["of the"]))
print
print ("The reason for these drastic changes is that these bigrams have to 'distribute their count' amongst many bigrams.")
print ("Furthermore, in one add smoothing, the bigrams which have higher counts are affected the most.")

Three examples of bigrams whose change in count were drastic
1. <s>,i
Original count = 121
Count after smoothing = 32.0812595226

2. said the
Original count = 209
Count after smoothing = 21.370741483

3. of the
Original count = 133
Count after smoothing = 14.560461285

The reason for these drastic changes is that these bigrams have to 'distribute their count' amongst many bigrams.
Furthermore, in one add smoothing, the bigrams which have higher counts are affected the most.


In [54]:
add_one_p = {}
for i in add_one_count:
    w1 = i.split()[0]
    if w1 not in tr_u_c:
        add_one_p[i] = add_one_count[i]
    else:
        add_one_p[i] = float(add_one_count[i])/tr_u_c[w1]
        

## Task 6

In [65]:
#6 part 1 (getting the counts for those bigrams whose original count was between 0 and 10)

c = tr_b_c.values()
c = collections.Counter(c)
c[0] = ((len(u_c)*len(u_c)) - len(tr_b_c)) - len(u_c)  #len(u_c) is subtracted once because we are not considering 
                                                       #the bigrams which start with </s> (as that can't happen)
c_new = {}
for i in range(11):
    c_new[i] = (c[i+1]*(i+1.0))/c[i]
    
diff = []
c_new_val = []
print ("Count\tDiscounting value")
for i in range(11):
    print (str(i) + "\t" + str(i - c_new[i]))
    diff.append(i-c_new[i])
    c_new_val.append(c_new[i])
    
d = sum(diff[1:9])/8

print 
print ("We see that the discounting value is fairly constant in the range 1-8 (with a few exceptions).")
print ("The average value for d in this range = "+ str(d))
print
print ("Note - Since the smoothing has been done only on the train data, this value can change based on how the corpus is divided into train and test data")

Count	Discounting value
0	-0.00135758789299
1	0.680661440996
2	0.860121049092
3	1.16637168142
4	1.04633204633
5	0.372549019608
6	1.07627118644
7	1.69879518072
8	1.12727272727
9	2.33333333333
10	-1.78571428571

We see that the discounting value is fairly constant in the range 1-8 (with a few exceptions).
The average value for d in this range = 1.00354679149

Note - Since the smoothing has been done only on the train data, this value can change based on how the corpus is divided into train and test data


In [68]:
#6 part 2 (extrapolating the above values for the remaining bigrams)
from scipy.optimize import curve_fit

def curve(n,k,b):
     return k*(n**b)
    
popt, pcov = curve_fit(curve, list(range(2,11)), c_new_val[2:11])
k = popt[0]
b = popt[1]

    

In [56]:
#6 part 3 (creating the dictionary which holds the counts of the bigrams)
good_tur_count = {}
for i in u_c:
    for j in u_c:
        if i == '</s>':  #because a bigram cannot start with </s> (end of sentence token)
            break
        if i + ' ' + j not in tr_b_c:
            good_tur_count[i + ' ' + j] = c_new[0]
        else:
            if 1<= tr_b_c[i + ' ' + j] <= 10:
                good_tur_count[i + ' ' + j] = c_new[tr_b_c[i + ' ' + j]]
            else:
                good_tur_count[i + ' ' + j] = k*(tr_b_c[i + ' ' + j]**b)


In [57]:
good_tur_p = {}
for i in good_tur_count:
    w1 = i.split()[0]
    if w1 not in tr_u_c:
        good_tur_p[i] = good_tur_count[i]
    else:
        good_tur_p[i] = float(good_tur_count[i])/tr_u_c[w1]
        

## Task 7

In [58]:
#7 Finding perpexity using the test set
# For add one smoothing
n = 0
add_one_per = 1.0
for i in test:
    l = i.split()
    n+= len(l) - 1

        
for i in test:
    l = i.split()
    for j in range(len(l)-1):
        k = (1/add_one_p[l[j] + ' ' + l[j+1]])
        
        add_one_per *=  math.pow(k,1.0/n)


In [60]:
#for good turing smoothing
n = 0
good_tur_per = 1.0
for i in test:
    l = i.split()
    n+= len(l) - 1

        
for i in test:
    l = i.split()
    for j in range(len(l)-1):
        k = (1/good_tur_p[l[j] + ' ' + l[j+1]])
        
        good_tur_per *=  math.pow(k,1.0/n)

In [66]:
print ("Perplexity of bigram model for test data after one add smoothing = " + str(add_one_per))
print ("Perplexity of bigram model for test data after good turing smoothing = " + str(good_tur_per))
print
print ("Since the perplexity of the good turing smoothing is less than that of the one add smoothing, we can say that Good Turing smoothing performs better.")

Perplexity of bigram model for test data after one add smoothing = 861.80604257
Perplexity of bigram model for test data after good turing smoothing = 349.505943307

Since the perplexity of the good turing smoothing is less than that of the one add smoothing, we can say that Good Turing smoothing performs better.
